In [2]:
!pip install pandas numpy scikit-learn nltk spacy
!python -m spacy download en_core_web_sm


  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached spacy-3.8.11-cp313-cp313-win_amd64.whl.metadata (28 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   -------------------- ------------------- 0.8/1.5 MB 2.2 MB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.5 MB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.2 MB 3.6 MB/s eta 0:00:04
   ---- ----------------------------------- 1.6/14.2 MB 3.9 MB/s eta 0:00:04
   ------ --------------------------------- 2.4/14.2 MB 4.1 MB/s eta 0:00:03
   -------- ------------------------------- 2.9/14.2 MB 4.1 MB/s eta 0:00:03
   ---------- ----------------------------- 3.7/14.2 MB 3.6 MB

In [3]:
import pandas as pd

# Sample Q&A dataset
data = {
    "Question": [
        "What are the benefits of exercise?",
        "Explain the water cycle."
    ],
    "Expected_Answer": [
        "Exercise improves cardiovascular health, strengthens muscles, and boosts mood.",
        "The water cycle involves evaporation, condensation, precipitation, and collection."
    ],
    "Student_Answer": [
        "Exercise strengthens muscles and improves mood.",
        "Water evaporates and then rains back."
    ]
}

df = pd.DataFrame(data)
df


,Question,Expected_Answer,Student_Answer
0,What are the benefits of exercise?,"Exercise improves cardiovascular health, stren...",Exercise strengthens muscles and improves mood.
1,Explain the water cycle.,"The water cycle involves evaporation, condensa...",Water evaporates and then rains back.


In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_keywords(text):
    doc = nlp(text.lower())
    keywords = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    return keywords

df["Expected_Keywords"] = df["Expected_Answer"].apply(extract_keywords)
df["Student_Keywords"] = df["Student_Answer"].apply(extract_keywords)
df


,Question,Expected_Answer,Student_Answer,Expected_Keywords,Student_Keywords
0,What are the benefits of exercise?,"Exercise improves cardiovascular health, stren...",Exercise strengthens muscles and improves mood.,"[exercise, improve, cardiovascular, health, st...","[exercise, strengthen, muscle, improve, mood]"
1,Explain the water cycle.,"The water cycle involves evaporation, condensa...",Water evaporates and then rains back.,"[water, cycle, involve, evaporation, condensat...","[water, evaporate, rain]"


In [5]:
def detect_gap(expected, student):
    missing = [word for word in expected if word not in student]
    return missing

df["Knowledge_Gap"] = df.apply(lambda x: detect_gap(x["Expected_Keywords"], x["Student_Keywords"]), axis=1)
df


,Question,Expected_Answer,Student_Answer,Expected_Keywords,Student_Keywords,Knowledge_Gap
0,What are the benefits of exercise?,"Exercise improves cardiovascular health, stren...",Exercise strengthens muscles and improves mood.,"[exercise, improve, cardiovascular, health, st...","[exercise, strengthen, muscle, improve, mood]","[cardiovascular, health, boost]"
1,Explain the water cycle.,"The water cycle involves evaporation, condensa...",Water evaporates and then rains back.,"[water, cycle, involve, evaporation, condensat...","[water, evaporate, rain]","[cycle, involve, evaporation, condensation, pr..."
